# Storing and Loading Models

https://pytorch.org/tutorials/beginner/saving_loading_models.html

## Init, helpers, utils, ...

In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
import matplotlib.pyplot as plt
from pprint import pprint
import numpy as np
from IPython.core.debugger import set_trace

import utils  # little helpers

# `state_dict()`

## `nn.Module.state_dict()`
`nn.Module` contain state dict, that maps each layer to the learnable parameters.

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
model = Net()

In [ ]:
model.state_dict()

In [ ]:
def state_dict_info(obj):
    print(f"{'layer':25} shape")
    print("===================================================")
    for k,v in obj.state_dict().items():
        try:
            print(f"{k:25} {v.shape}")
        except AttributeError:
            print(f"{k:25} {v}")

In [ ]:
state_dict_info(model)

## `nn.Optimizer`

Optimizers also have a a `state_dict`.

In [ ]:
optimizer = optim.Adadelta(model.parameters())

In [ ]:
state_dict_info(optimizer)

In [ ]:
optimizer.state_dict()["state"]

In [ ]:
optimizer.state_dict()["param_groups"]

## Storing and loading `state_dict`

In [ ]:
model_file = "model_state_dict.pt"
torch.save(model.state_dict(), model_file)

In [ ]:
model = Net()
model.load_state_dict(torch.load(model_file))

## Storing and loading the full model

In [ ]:
model_file = "model_123.pt"
torch.save(model, model_file)

In [ ]:
# Only works if code for `Net` is available right now
model = torch.load(model_file)

# Example Checkpointing
You can store model, optimizer and arbitrary information and reload it.

Example:
```python
torch.save(
    {
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'epoch': epoch,
        'loss': loss,
    },
    PATH,
)
```

# Exercise
- Find out what is going to be in the `state` variable of the `state_dict` of an optimizer.
- Write your own checkpoint functionality.